In [19]:
import pandas as pd
import numpy as np
import tweepy as tw
import jsonlines
import math
import pickle5
np.set_printoptions(suppress=True)
from fn import getRetweetsFavsFollowersInfo, getTagsMediaHashtagsInfo

# Analysis

In [21]:
# import news dataset
df_true = pd.read_pickle("true_roots_df_final.pkl")
df_fake = pd.read_pickle("fake_roots_df_final.pkl")

In [22]:
# import Shahi datasets
df_shahi_I = pd.read_pickle("shahi_dataset-I.pkl")
df_shahi_II = pd.read_pickle("shahi_dataset-II.pkl")
df_shahi_II_old = pd.read_pickle("shahi_dataset-II_old.pkl")
df_shahi_II_large = pd.read_pickle("shahi_dataset-II_v2.pkl")

In [23]:
# import Patwa dataset
f = jsonlines.open("Patwa Text.jsonl",)
for line in f.iter():
    patwa_all = line
patwa_real_text = patwa_all['real_text']
patwa_fake_text = patwa_all['fake_text']

In [24]:
# import ArCOV dataset
df_ArCOV_real = pd.read_pickle("ArCOV_real.pkl")
df_ArCOV_fake = pd.read_pickle("ArCOV_fake.pkl")

In [25]:
# import CHECKED dataset
df_checked_real = pd.read_pickle("checked_real.pkl")
df_checked_fake = pd.read_pickle("checked_fake.pkl")

In [26]:
# import Emily Chen's datasets
with open("viral_OG_tweets_5000.pkl", "rb") as f:
    viral_tweets = pickle5.load(f)
viral_tweets = viral_tweets[(viral_tweets['date'] < '2021-02-12')]
viral_tweets = viral_tweets.sort_values(['#Retweets'], ascending = False).reset_index()
df_chen_viral = viral_tweets.head(100)
df_chen_roots = pd.read_pickle("Chen_sample_roots.pkl")
df_chen_skipRTs = pd.read_pickle("Chen_sample_skipRTs.pkl")
df_chen_skipRTs_100 = pd.read_pickle("Chen_sample_skipRTs_100.pkl")

In [29]:
df_chen_skipRTs_100

,level_0,index,tweetID,date,text,isRT,#Retweets,#Favorites,user_name,user_date,...,mentions,URLs,media,replyID,replyUser,replyName,sens,lang,OGtweetID,OGdate
0,94,255,1356323819363819520,2021-02-01 19:29:32,VENEZIA / Il carnevale triste delle saracinesc...,False,1,1,nuova_venezia,Wed Dec 16 11:27:19 +0000 2009,...,[],"[{'url': 'https://t.co/bfeccr0KIn', 'expanded_...",None,9999999999999999999,9999999999999999999,None,None,it,9999999999999999999,None
1,92,252,1356418616493768710,2021-02-02 01:46:13,Das Jahr hat verdammt schlecht begonnen und ic...,False,0,0,andibaaaa,Sun Apr 12 07:34:16 +0000 2020,...,[],[],None,9999999999999999999,9999999999999999999,None,None,de,9999999999999999999,None
2,70,176,1356555574872309761,2021-02-02 10:50:27,@emmadentcoad Unlike Labour to try and politic...,False,1,3,HonestCitizen62,Sat Jun 03 18:21:33 +0000 2017,...,"[{'screen_name': 'emmadentcoad', 'name': 'Emma...",[],None,1355931192474931204,244489735,emmadentcoad,None,en,9999999999999999999,None
3,88,227,1356769590123126786,2021-02-03 01:00:52,Corona-Impfungen in Deutschland: Fast zwei Mil...,False,0,0,Allergotrade,Tue May 14 22:13:44 +0000 2013,...,[],"[{'url': 'https://t.co/H8CzuUvxaI', 'expanded_...",None,9999999999999999999,9999999999999999999,None,None,de,9999999999999999999,None
4,87,226,1356892962458906624,2021-02-03 09:11:06,@derrick_4realz @WeldonCaleb @Hunter_3MRP @Pub...,False,0,0,PackerHasan,Mon Mar 16 18:32:57 +0000 2015,...,"[{'screen_name': 'derrick_4realz', 'name': 'An...",[],None,1356735722095431683,1972456604,derrick_4realz,None,en,9999999999999999999,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,93,254,1365918171627069444,2021-02-28 06:54:04,@ULucq https://t.co/rK58RNUapE,False,0,0,_fadhilmy,Sun May 20 22:27:51 +0000 2018,...,"[{'screen_name': 'ULucq', 'name': 'Uncle Lucq'...","[{'url': 'https://t.co/rK58RNUapE', 'expanded_...",None,1365917720076718081,1309121049242591233,ULucq,None,und,9999999999999999999,None
96,2,9,1365975935556718600,2021-02-28 10:43:36,@stephenfry How about this one. Covid tax on a...,False,0,0,JohnOmtp,Sun May 01 09:02:02 +0000 2011,...,"[{'screen_name': 'stephenfry', 'name': 'Stephe...",[],None,1365722832009797635,15439395,stephenfry,None,en,9999999999999999999,None
97,45,133,1366006784461312001,2021-02-28 12:46:11,Now@cp24.com mr mayor why is it soccer can be ...,False,0,0,sherrard27,Thu Mar 19 22:28:37 +0000 2015,...,[],[],None,9999999999999999999,9999999999999999999,None,None,en,9999999999999999999,None
98,44,129,1366086015656599564,2021-02-28 18:01:01,Seattle Times: A pandemic-era rise in clamming...,False,0,0,buzz_corona,Sat Mar 14 14:13:07 +0000 2020,...,[],"[{'url': 'https://t.co/AVwBO4CdR3', 'expanded_...",None,9999999999999999999,9999999999999999999,None,None,en,9999999999999999999,None


## Retweets, Favorites, and Followers

In [9]:
def calcPropRetweet(df):
    return len(df[df['isRT'] == True]) / len(df)

In [10]:
def getArrayDivisionPercentile(df_col1, df_col2, percentile):
    divisions = (df_col1/df_col2).tolist()
    count = 0
    while (count < len(divisions)):
        if math.isnan(divisions[count]):
            divisions[count] = 0
        if math.isinf(divisions[count]):
            del divisions[count]
        else:
            count += 1
    return np.percentile(divisions,percentile)

In [11]:
def getRetweetsFavsFollowersInfo(df, dataset_name):
    n = len(df)
    avg_RT = np.mean(df['#Retweets'])
    avg_favs = np.mean(df['#Favorites'])
    RTsToFavs = avg_RT/avg_favs
    RTsToFavs_75 = getArrayDivisionPercentile(df['#Retweets'],df['#Favorites'], 75)
    if ('#Followers' and 'verified' in df):
        prop_RT = calcPropRetweet(df)
        avg_fols = np.mean(df['#Followers'])
        med_fols = np.median(df['#Followers'])
        RTsToFols = avg_RT/avg_fols
        RTsToFols_75 = getArrayDivisionPercentile(df['#Retweets'],df['#Followers'], 75)
        prop_verified = sum(df['verified'])/n
    else:
        prop_RT = 'N/A'
        avg_fols = 'N/A'
        med_fols = 'N/A'
        RTsToFols = 'N/A'
        RTsToFols_75 = 'N/A'
        prop_verified = 'N/A'
    if ('#Comments' in df):
        avg_comments = np.mean(df['#Comments'])
    else:
        avg_comments = 'N/A'
    data = [[dataset_name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified]]
    result = pd.DataFrame(data, columns = ['Name', 'n','prop_RT','avg_RT','avg_favs','avg_comments','med_fols','avg_fols','RTsToFols','RTsToFols_75','RTsToFavs','RTsToFavs_75','prop_verified'])
    return result

In [12]:
tf_data = getRetweetsFavsFollowersInfo(df_true, 'True News').append(getRetweetsFavsFollowersInfo(df_fake, 'Fake News'))
tf_data

,Name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified
0,True News,65,0.0,88.723077,809.123077,N/A,5289.0,109544.723077,0.000810,0.000034,0.109653,0.286284,0.400000
0,Fake News,67,0.0,508.179104,1434.940299,N/A,1008.0,145289.671642,0.003498,0.005963,0.354147,0.447432,0.208955


In [14]:
shahi_data = getRetweetsFavsFollowersInfo(df_shahi_II_old, 'Shahi II_old').append(getRetweetsFavsFollowersInfo(df_shahi_I, 'Shahi I'))
getRetweetsFavsFollowersInfo(df_shahi_II_old, 'Shahi II_old').append(getRetweetsFavsFollowersInfo(df_shahi_II, 'Shahi II')).append(getRetweetsFavsFollowersInfo(df_shahi_II_large, 'Shahi II_large')).append(getRetweetsFavsFollowersInfo(df_shahi_I, 'Shahi I'))

,Name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified
0,Shahi II_old,764,0.045812,418.291885,315.272251,N/A,1408.0,1.578650e+05,0.002650,0.002913,1.326764,0.385236,0.115183
0,Shahi II,673,0.315007,4044.197623,305.344725,N/A,915.0,1.700271e+05,0.023786,0.047953,13.244695,0.363757,0.099554
0,Shahi II_large,4464,0.306228,3464.788754,253.159722,N/A,968.5,2.556090e+05,0.013555,0.056726,13.686177,0.400000,0.112903
0,Shahi I,741,0.078273,3453.137652,7824.434548,N/A,56466.0,1.687189e+06,0.002047,0.010726,0.441327,0.659433,0.531714


In [15]:
arcov_data = getRetweetsFavsFollowersInfo(df_ArCOV_real, 'ArCOV Real').append(getRetweetsFavsFollowersInfo(df_ArCOV_fake, 'ArCOV Fake'))
arcov_data

,Name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified
0,ArCOV Real,1645,0.0,12.743465,34.308207,N/A,17584.0,473207.135562,0.000027,0.000123,0.371441,0.500000,0.341033
0,ArCOV Fake,1380,0.0,5.576087,16.414493,N/A,1306.5,78206.460145,0.000071,0.000436,0.339705,0.333333,0.061594


In [16]:
checked_data = getRetweetsFavsFollowersInfo(df_checked_real, 'CHECKED Real').append(getRetweetsFavsFollowersInfo(df_checked_fake, 'CHECKED Fake'))
checked_data

,Name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified
0,CHECKED Real,1772,N/A,4122.170429,31690.484763,2110.939052,N/A,N/A,N/A,N/A,0.130076,0.16953,N/A
0,CHECKED Fake,338,N/A,164.213018,1314.295858,111.180473,N/A,N/A,N/A,N/A,0.124944,0.21875,N/A


In [17]:
chen_data = getRetweetsFavsFollowersInfo(df_chen_roots, 'Chen Roots').append(getRetweetsFavsFollowersInfo(df_chen_skipRTs_100, 'Chen skipRTs 100')).append(getRetweetsFavsFollowersInfo(df_chen_skipRTs, 'Chen skipRTs')).append(getRetweetsFavsFollowersInfo(df_chen_viral, 'Chen Viral'))
chen_data

,Name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified
0,Chen Roots,92,0.0,526.260870,2008.23913,N/A,5838.0,199413.978261,0.002639,0.007816,0.262051,0.500000,0.228261
0,Chen skipRTs 100,100,0.0,0.360000,1.15000,N/A,272.5,7644.480000,0.000047,0.000000,0.313043,0.000000,0.040000
0,Chen skipRTs,1130,0.0,0.815929,3.99292,N/A,461.0,41550.851327,0.000020,0.000000,0.204344,0.000000,0.048673
0,Chen Viral,100,0.0,40898.270000,214382.31000,N/A,8110.0,493222.080000,0.082921,15.601794,0.190773,0.308618,0.260000


In [18]:
tf_data.append(shahi_data).append(arcov_data).append(checked_data).append(chen_data)

,Name,n,prop_RT,avg_RT,avg_favs,avg_comments,med_fols,avg_fols,RTsToFols,RTsToFols_75,RTsToFavs,RTsToFavs_75,prop_verified
0,True News,65,0.0,88.723077,809.123077,N/A,5289.0,109544.723077,0.00081,0.000034,0.109653,0.286284,0.4
0,Fake News,67,0.0,508.179104,1434.940299,N/A,1008.0,145289.671642,0.003498,0.005963,0.354147,0.447432,0.208955
0,Shahi II_old,764,0.045812,418.291885,315.272251,N/A,1408.0,157865.01178,0.00265,0.002913,1.326764,0.385236,0.115183
0,Shahi I,741,0.078273,3453.137652,7824.434548,N/A,56466.0,1687189.325236,0.002047,0.010726,0.441327,0.659433,0.531714
0,ArCOV Real,1645,0.0,12.743465,34.308207,N/A,17584.0,473207.135562,0.000027,0.000123,0.371441,0.500000,0.341033
0,ArCOV Fake,1380,0.0,5.576087,16.414493,N/A,1306.5,78206.460145,0.000071,0.000436,0.339705,0.333333,0.061594
0,CHECKED Real,1772,N/A,4122.170429,31690.484763,2110.939052,N/A,N/A,N/A,N/A,0.130076,0.169530,N/A
0,CHECKED Fake,338,N/A,164.213018,1314.295858,111.180473,N/A,N/A,N/A,N/A,0.124944,0.218750,N/A
0,Chen Roots,92,0.0,526.260870,2008.239130,N/A,5838.0,199413.978261,0.002639,0.007816,0.262051,0.500000,0.228261
0,Chen skipRTs 100,100,0.0,0.360000,1.150000,N/A,272.5,7644.48,0.000047,0.0,0.313043,0.000000,0.04


## User mentions, Media, Hashtags

In [12]:
getTagsMediaHashtagsInfo(df_true, "True News").append(getTagsMediaHashtagsInfo(df_fake, "Fake News"))

,Name,n,prop_tagged,avg_num_tags,prop_media,prop_pic,prop_video,prop_hashtagged,avg_num_hashtags
0,True News,65,0.200000,1.000000,0.123077,N/A,N/A,0.092308,2.333333
0,Fake News,67,0.522388,4.885714,0.283582,N/A,N/A,0.074627,1.000000


In [13]:
getTagsMediaHashtagsInfo(df_shahi_II, "Shahi II").append(getTagsMediaHashtagsInfo(df_shahi_I, "Shahi I"))

,Name,n,prop_tagged,avg_num_tags,prop_media,prop_pic,prop_video,prop_hashtagged,avg_num_hashtags
0,Shahi II,764,0.439791,2.491071,0.267016,N/A,N/A,0.215969,2.242424
0,Shahi I,741,0.357625,1.735849,0.596491,N/A,N/A,0.377868,2.132143


In [14]:
getTagsMediaHashtagsInfo(df_ArCOV_real, 'ArCOV Real').append(getTagsMediaHashtagsInfo(df_ArCOV_fake, 'ArCOV Fake'))

,Name,n,prop_tagged,avg_num_tags,prop_media,prop_pic,prop_video,prop_hashtagged,avg_num_hashtags
0,ArCOV Real,1645,0.184195,1.399340,0.516717,N/A,N/A,0.798784,2.769406
0,ArCOV Fake,1380,0.101449,1.185714,0.470290,N/A,N/A,0.706522,2.175385


In [15]:
getTagsMediaHashtagsInfo(df_checked_real, 'CHECKED Real').append(getTagsMediaHashtagsInfo(df_checked_fake, 'CHECKED Fake'))

,Name,n,prop_tagged,avg_num_tags,prop_media,prop_pic,prop_video,prop_hashtagged,avg_num_hashtags
0,CHECKED Real,1772,N/A,N/A,0.968962,0.650113,0.318849,N/A,N/A
0,CHECKED Fake,338,N/A,N/A,0.461538,0.156805,0.304734,N/A,N/A


In [30]:
getTagsMediaHashtagsInfo(df_chen_skipRTs, 'Chen skipRTs').append(getTagsMediaHashtagsInfo(df_chen_viral, 'Chen Viral'))

,Name,n,prop_tagged,avg_num_tags,prop_media,prop_pic,prop_video,prop_hashtagged,avg_num_hashtags
0,Chen skipRTs,1130,0.40531,1.921397,0.130088,N/A,N/A,0.223009,2.809524
0,Chen Viral,100,0.01000,1.000000,0.240000,N/A,N/A,0.030000,1.333333
